# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://vnexpress.net/bat-dong-san")
ed.links

['javascript:;',
 '/',
 '/tin-tuc-24h',
 'https://vnexpress.net/topic/ha-noi-26482',
 'https://vnexpress.net/topic/tp-ho-chi-minh-26483',
 'https://e.vnexpress.net/',
 '/myvne',
 '/',
 '/tin-tuc-24h',
 '/thoi-su',
 '/goc-nhin',
 '/the-gioi',
 'https://video.vnexpress.net',
 '/podcast',
 '/kinh-doanh',
 '/bat-dong-san',
 '/khoa-hoc',
 '/giai-tri',
 '/the-thao',
 '/phap-luat',
 '/giao-duc',
 '/suc-khoe',
 '/doi-song',
 '/du-lich',
 '/cong-nghe',
 '/oto-xe-may',
 '/y-kien',
 '/tam-su',
 'javascript:;',
 '/bat-dong-san',
 '/bat-dong-san/chinh-sach',
 '/bat-dong-san/thi-truong',
 '/bat-dong-san/du-an',
 '/bat-dong-san/khong-gian-song',
 '/bat-dong-san/tu-van',
 'https://vnexpress.net/dkra-giao-dich-condotel-tang-gan-3-lan-trong-nam-ngoai-4839057.html',
 'https://vnexpress.net/dkra-giao-dich-condotel-tang-gan-3-lan-trong-nam-ngoai-4839057.html',
 'https://vnexpress.net/dkra-giao-dich-condotel-tang-gan-3-lan-trong-nam-ngoai-4839057.html',
 'https://vnexpress.net/dkra-giao-dich-condotel-tang-g

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about lastest news, \
such as links to Breaking news, current events, press, reporter, journalist, article, news story, headline, editorial, fake news, front page, mainstream media, tabloid, broadsheet, credibility, censorship, propaganda, media outlet, live coverage, ratings.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/real estate"},
        {"type": "careers page": "url": "https://another.full.url/news"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about lastest news, such as links to Breaking news, current events, press, reporter, journalist, article, news story, headline, editorial, fake news, front page, mainstream media, tabloid, broadsheet, credibility, censorship, propaganda, media outlet, live coverage, ratings.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/real estate"},
        {"type": "careers page": "url": "https://another.full.url/news"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://vnexpress.net/bat-dong-san - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
javascript:;
/
/tin-tuc-24h
https://vnexpress.net/topic/ha-noi-26482
https://vnexpress.net/topic/tp-ho-chi-minh-26483
https://e.vnexpress.net/
/myvne
/
/tin-tuc-24h
/thoi-su
/goc-nhin
/the-gioi
https://video.vnexpress.net
/podcast
/kinh-doanh
/bat-dong-san
/khoa-hoc
/giai-tri
/the-thao
/phap-luat
/giao-duc
/suc-khoe
/doi-song
/du-lich
/cong-nghe
/oto-xe-may
/y-kien
/tam-su
javascript:;
/bat-dong-san
/bat-dong-san/chinh-sach
/bat-dong-san/thi-truong
/bat-dong-san/du-an
/bat-dong-san/khong-gian-song
/bat-dong-san/tu-van
https://vnexpress.net/dkra-giao-dich-condotel-tang-gan-3-lan-trong-nam-ngoai-4839057.html
https://vnexpress.net/dkra-giao-dich-condotel-tang-gan-3-lan-trong-nam-ngoai-4

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://vnexpress.net/bat-dong-san")
huggingface.links

['javascript:;',
 '/',
 '/tin-tuc-24h',
 'https://vnexpress.net/topic/ha-noi-26482',
 'https://vnexpress.net/topic/tp-ho-chi-minh-26483',
 'https://e.vnexpress.net/',
 '/myvne',
 '/',
 '/tin-tuc-24h',
 '/thoi-su',
 '/goc-nhin',
 '/the-gioi',
 'https://video.vnexpress.net',
 '/podcast',
 '/kinh-doanh',
 '/bat-dong-san',
 '/khoa-hoc',
 '/giai-tri',
 '/the-thao',
 '/phap-luat',
 '/giao-duc',
 '/suc-khoe',
 '/doi-song',
 '/du-lich',
 '/cong-nghe',
 '/oto-xe-may',
 '/y-kien',
 '/tam-su',
 'javascript:;',
 '/bat-dong-san',
 '/bat-dong-san/chinh-sach',
 '/bat-dong-san/thi-truong',
 '/bat-dong-san/du-an',
 '/bat-dong-san/khong-gian-song',
 '/bat-dong-san/tu-van',
 'https://vnexpress.net/dkra-giao-dich-condotel-tang-gan-3-lan-trong-nam-ngoai-4839057.html',
 'https://vnexpress.net/dkra-giao-dich-condotel-tang-gan-3-lan-trong-nam-ngoai-4839057.html',
 'https://vnexpress.net/dkra-giao-dich-condotel-tang-gan-3-lan-trong-nam-ngoai-4839057.html',
 'https://vnexpress.net/dkra-giao-dich-condotel-tang-g

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'homepage', 'url': 'https://huggingface.co'},
  {'type': 'models', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets', 'url': 'https://huggingface.co/datasets'},
  {'type': 'spaces', 'url': 'https://huggingface.co/spaces'},
  {'type': 'posts', 'url': 'https://huggingface.co/posts'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs'},
  {'type': 'enterprise', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing', 'url': 'https://huggingface.co/pricing'},
  {'type': 'chat', 'url': 'https://huggingface.co/chat'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'status', 'url': 'https://status.huggingface.co'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://vnexpress.net/bat-dong-san"))

Found links: {'links': [{'type': 'news article', 'url': 'https://vnexpress.net/dkra-giao-dich-condotel-tang-gan-3-lan-trong-nam-ngoai-4839057.html'}, {'type': 'news article', 'url': 'https://vnexpress.net/shark-hung-hat-nhay-tai-tiec-cuoi-nam-cen-group-4840763.html'}, {'type': 'news article', 'url': 'https://vnexpress.net/khu-do-thi-phu-my-hung-xay-lan-400-ha-so-voi-quy-hoach-4840752.html'}, {'type': 'news article', 'url': 'https://vnexpress.net/tp-hcm-se-co-them-12-du-an-nha-o-xa-hoi-4840817.html'}, {'type': 'news article', 'url': 'https://vnexpress.net/dong-tien-dau-tu-bat-dong-san-dang-tro-lai-mien-nam-4839030.html'}, {'type': 'news article', 'url': 'https://vnexpress.net/can-ho-77-m2-mau-hong-mang-tinh-than-thoi-trang-4840544.html'}, {'type': 'news article', 'url': 'https://vnexpress.net/ha-noi-co-44-du-an-nha-thuong-mai-chuan-bi-duoc-dau-tu-4840378.html'}, {'type': 'news article', 'url': 'https://vnexpress.net/gia-thue-nha-o-los-angeles-tang-vot-sau-chay-rung-4840465.html'}, {'typ

In [14]:
system_prompt = "You are an experienced journalist and professional writer who excels at analyzing and synthesizing \
information into compelling news stories - analyze multiple sources carefully, identify the most newsworthy and valuable \
information, present findings in clear journalistic style using inverted pyramid structure with compelling headlines, strong leads, \
organized content, relevant quotes/data, and proper attribution while maintaining high standards of accuracy, \
objectivity and reader engagement - respond in markdown format including proper formatting, subheadings and AP style guidelines throughout the article. Return results Vienamese"

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [29]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at many news called: {company_name}\n"
    user_prompt += f"Here are the contents from multiple verified sources; synthesize this information into a compelling news story with fact-based reporting, key developments, expert insights and relevant context - deliver the article in AP style using markdown format to ensure maximum value and clarity for readers.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:10_000] # Truncate if more than 5,000 characters
    return user_prompt

In [30]:
get_brochure_user_prompt("vnexpress", "https://vnexpress.net/bat-dong-san")

Found links: {'links': [{'type': 'news article', 'url': 'https://vnexpress.net/dkra-giao-dich-condotel-tang-gan-3-lan-trong-nam-ngoai-4839057.html'}, {'type': 'news article', 'url': 'https://vnexpress.net/shark-hung-hat-nhay-tai-tiec-cuoi-nam-cen-group-4840763.html'}, {'type': 'news article', 'url': 'https://vnexpress.net/khu-do-thi-phu-my-hung-xay-lan-400-ha-so-voi-quy-hoach-4840752.html'}, {'type': 'news article', 'url': 'https://vnexpress.net/tp-hcm-se-co-them-12-du-an-nha-o-xa-hoi-4840817.html'}, {'type': 'news article', 'url': 'https://vnexpress.net/dong-tien-dau-tu-bat-dong-san-dang-tro-lai-mien-nam-4839030.html'}, {'type': 'news article', 'url': 'https://vnexpress.net/can-ho-77-m2-mau-hong-mang-tinh-than-thoi-trang-4840544.html'}, {'type': 'news article', 'url': 'https://vnexpress.net/ha-noi-co-44-du-an-nha-thuong-mai-chuan-bi-duoc-dau-tu-4840378.html'}, {'type': 'news article', 'url': 'https://vnexpress.net/gia-thue-nha-o-los-angeles-tang-vot-sau-chay-rung-4840465.html'}, {'typ

'You are looking at many news called: vnexpress\nHere are the contents from multiple verified sources; synthesize this information into a compelling news story with fact-based reporting, key developments, expert insights and relevant context - deliver the article in AP style using markdown format to ensure maximum value and clarity for readers.\nLanding page:\nWebpage Title:\nBất động sản - Tin tức thị trường, dự án, đầu tư | VnExpress\nWebpage Contents:\nThứ hai, 20/1/2025\nMới nhất\nTin theo khu vực\nHà Nội\nTP Hồ Chí Minh\nInternational\nMới nhất\nThời sự\nGóc nhìn\nThế giới\nVideo\nPodcasts\nKinh doanh\nBất động sản\nKhoa học\nGiải trí\nThể thao\nPháp luật\nGiáo dục\nSức khỏe\nĐời sống\nDu lịch\nCông nghệ\nXe\nÝ kiến\nTâm sự\nTất cả\nBất động sản\nChính sách\nThị trường\nDự án\nKhông gian sống\nTư vấn\nDKRA: Giao dịch condotel tăng gần 3 lần trong năm ngoái\nNăm ngoái, lượng tiêu thụ condotel sơ cấp đạt gần 3.100 căn, tăng 2,6 lần so cùng kỳ song chỉ tập trung ở một số khu vực, the

In [31]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [32]:
create_brochure("vnexpress", "https://vnexpress.net/bat-dong-san")

Found links: {'links': [{'type': 'news article', 'url': 'https://vnexpress.net/dkra-giao-dich-condotel-tang-gan-3-lan-trong-nam-ngoai-4839057.html'}, {'type': 'news article', 'url': 'https://vnexpress.net/shark-hung-hat-nhay-tai-tiec-cuoi-nam-cen-group-4840763.html'}, {'type': 'news article', 'url': 'https://vnexpress.net/khu-do-thi-phu-my-hung-xay-lan-400-ha-so-voi-quy-hoach-4840752.html'}, {'type': 'news article', 'url': 'https://vnexpress.net/tp-hcm-se-co-them-12-du-an-nha-o-xa-hoi-4840817.html'}, {'type': 'news article', 'url': 'https://vnexpress.net/dong-tien-dau-tu-bat-dong-san-dang-tro-lai-mien-nam-4839030.html'}, {'type': 'news article', 'url': 'https://vnexpress.net/can-ho-77-m2-mau-hong-mang-tinh-than-thoi-trang-4840544.html'}, {'type': 'news article', 'url': 'https://vnexpress.net/ha-noi-co-44-du-an-nha-thuong-mai-chuan-bi-duoc-dau-tu-4840378.html'}, {'type': 'news article', 'url': 'https://vnexpress.net/gia-thue-nha-o-los-angeles-tang-vot-sau-chay-rung-4840465.html'}, {'typ

```markdown
# Thị Trường Bất Động Sản Việt Nam: Những Diễn Biến Đáng Chú Ý Năm 2025

**HÀ NỘI, Việt Nam —** Thị trường bất động sản Việt Nam đang chứng kiến những diễn biến mạnh mẽ trong năm 2025, với sự gia tăng giao dịch, dòng tiền đầu tư trở lại miền Nam và những yêu cầu mới về chính sách nhằm điều chỉnh sự phát triển bền vững hơn.

## Tăng Trưởng Mạnh Mẽ Trong Phân Khúc Condotel

Theo báo cáo từ DKRA, lượng tiêu thụ condotel sơ cấp đã đạt gần 3.100 căn trong năm ngoái, tăng gần 2,6 lần so với năm 2023. Mặc dù con số này cho thấy sự phục hồi, nhưng sự tăng trưởng chủ yếu chỉ tập trung ở một số khu vực nhất định, làm cho thị trường này vẫn có sự phân hóa rõ nét.

## Dòng Tiền Đầu Tư Quay Về Miền Nam

Sau một năm "dạo chơi" tại miền Bắc, dòng tiền đầu tư đang chảy trở lại miền Nam, tạo sức sống mới cho thị trường bất động sản tại đây. Các nhà đầu tư đang tìm kiếm các dự án có tiềm năng sinh lời cao, đồng thời giải quyết tình trạng thiếu hụt nhà ở xã hội tại TP Hồ Chí Minh với dự kiến có thêm 12 dự án nhà ở xã hội trong năm 2025.

Ông Nguyễn Thế Hưng, chuyên gia bất động sản, nhận định: “Việc dòng tiền quay trở lại miền Nam cho thấy sự khởi sắc và kỳ vọng vào một năm đầy hứa hẹn cho thị trường bất động sản.”

## Những Vấn Đề Cần Giải Quyết Thông Qua Chính Sách Mới

Chính phủ Việt Nam đang xem xét các chính sách thuế mới nhằm hạn chế hoạt động đầu cơ bất động sản, theo chỉ đạo của Thủ tướng Phạm Minh Chính. Việc áp dụng các biện pháp thuế sẽ giúp cân bằng thị trường và giảm thiểu các rủi ro mà người mua nhà phải đối mặt, đặc biệt là các dự án giá rẻ, như khu căn hộ Legacy Alpha Valley tại Hà Nội, nơi có dấu hiệu rủi ro cao.

## Nhu Cầu Về Không Gian Sống Xanh Tăng Cao

Nhu cầu tìm kiếm môi trường sống xanh và đầy trải nghiệm ngày càng gia tăng, đặc biệt trong giới các gia đình trẻ có con nhỏ. Các dự án bất động sản tiện ích cao, như khu đô thị Sun Urban City, đang trở thành lựa chọn hàng đầu cho những ai muốn kết hợp giữa cuộc sống hiện đại và không gian sống gần gũi với thiên nhiên.

## Cơ Hội Đầu Tư Tại Khu Công Nghiệp

Năm 2025 được dự đoán sẽ là năm bùng nổ cho phân khúc bất động sản khu công nghiệp, với việc hàng loạt khu công nghiệp mới được chấp thuận đầu tư. Khu công nghiệp Bắc Đồng Phú tại Bình Phước đang được mở rộng với tổng nguồn vốn đầu tư lên đến 1.360 tỷ đồng.

## Kết Luận

Thị trường bất động sản Việt Nam năm 2025 sẽ tiếp tục chứng kiến sự tái cấu trúc về mặt chính sách, gia tăng nhu cầu về không gian sống xanh, và sự trở lại của dòng chảy đầu tư. Điều này không chỉ mang lại cơ hội cho các nhà đầu tư mà còn cho phép người mua nhà tìm kiếm căn hộ phù hợp với nhu cầu và tài chính của họ.
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [33]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [34]:
stream_brochure("HuggingFace", "https://vnexpress.net/bat-dong-san")

Found links: {'links': [{'type': 'current event', 'url': 'https://vnexpress.net/dkra-giao-dich-condotel-tang-gan-3-lan-trong-nam-ngoai-4839057.html'}, {'type': 'current event', 'url': 'https://vnexpress.net/shark-hung-hat-nhay-tai-tiec-cuoi-nam-cen-group-4840763.html'}, {'type': 'current event', 'url': 'https://vnexpress.net/khu-do-thi-phu-my-hung-xay-lan-400-ha-so-voi-quy-hoach-4840752.html'}, {'type': 'current event', 'url': 'https://vnexpress.net/tp-hcm-se-co-them-12-du-an-nha-o-xa-hoi-4840817.html'}, {'type': 'current event', 'url': 'https://vnexpress.net/dong-tien-dau-tu-bat-dong-san-dang-tro-lai-mien-nam-4839030.html'}, {'type': 'current event', 'url': 'https://vnexpress.net/can-ho-77-m2-mau-hong-mang-tinh-than-thoi-trang-4840544.html'}, {'type': 'current event', 'url': 'https://vnexpress.net/ha-noi-co-44-du-an-nha-thuong-mai-chuan-bi-duoc-dau-tu-4840378.html'}, {'type': 'current event', 'url': 'https://vnexpress.net/gia-thue-nha-o-los-angeles-tang-vot-sau-chay-rung-4840465.html'

# Sự Phục Hồi Của Thị Trường Bất Động Sản Việt Nam: Những Dấu Hiệu Tích Cực Trong Năm 2025

**Hà Nội, Việt Nam –** Trong bối cảnh thị trường bất động sản đã trải qua một năm nhiều biến động, các chuyên gia nhận định rằng năm 2025 sẽ đánh dấu sự trở lại của dòng tiền đầu tư, đặc biệt là khu vực miền Nam. Các báo cáo gần đây chỉ ra rằng số lượng căn hộ condotel tiêu thụ đã tăng gần ba lần trong năm qua, cho thấy một xu hướng tích cực.

## Tăng Trưởng Về Doanh Số Condotel

Theo báo cáo từ công ty DKRA, số lượng giao dịch căn hộ condotel sơ cấp đạt gần 3.100 căn trong năm 2024, gấp 2,6 lần so với cùng kỳ năm trước. Tuy nhiên, mức tăng trưởng này chỉ tập trung tại một số khu vực nhất định, cho thấy sự phân bổ không đồng đều trong sự phục hồi của thị trường.

> "Sự gia tăng này có thể cho thấy nhu cầu đang trở lại, nhất là trong bối cảnh các chính sách mới về phát triển nhà ở đang được áp dụng," chuyên gia từ DKRA cho biết.

## Thị Trường Nhà Ở Xã Hội Bùng Nổ Tại TP Hồ Chí Minh

Cũng trong năm 2025, TP Hồ Chí Minh dự kiến sẽ có thêm 12 dự án nhà ở xã hội mới. Theo Sở Xây dựng TP Hồ Chí Minh, 4 dự án sẽ hoàn thành và 8 dự án mới sẽ được khởi công, cho thấy quyết tâm cao của chính quyền trong việc giải quyết vấn đề nhà ở cho người dân.

## Khả Năng Tăng Giá Bất Động Sản

Bên cạnh sự phục hồi của giao dịch, giá bất động sản cũng có những dấu hiệu tăng trưởng. Tại Hà Nội, mảnh đất tại số 148 Giảng Võ mới được chuyển đổi mục đích sử dụng thành tổ hợp trung tâm thương mại, một bước đi cho thấy tiềm năng tăng giá trị bất động sản trong khu vực.

> "Chúng tôi ghi nhận rằng nhu cầu tìm kiếm không gian sống xanh và đầy trải nghiệm đang gia tăng, đặc biệt là từ các gia đình trẻ," một nhà nghiên cứu thị trường cho biết.

## Các Chính Sách Hỗ Trợ Đầu Tư

Thủ tướng Chính phủ Phạm Minh Chính đã yêu cầu Bộ Tài chính nghiên cứu đối với việc đánh thuế đầu cơ bất động sản, một động thái có thể tạo ra sự công bằng hơn cho thị trường và nhằm tăng cường hiệu quả hoạt động đầu tư.

## Tổng Kết

Sự phục hồi thị trường bất động sản Việt Nam đang diễn ra tại nhiều khu vực khác nhau. Với việc gia tăng các dự án nhà ở xã hội, sự trở lại của dòng tiền đầu tư cùng với chính sách của Chính phủ, ngành bất động sản hướng tới một năm 2025 đầy tiềm năng và cơ hội mới. Tuy nhiên, việc phân bổ nguồn lực và cải thiện hạ tầng vẫn là những thách thức đáng lưu ý cần phải giải quyết để đảm bảo sự phát triển bền vững cho thị trường này.

In [21]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'}, {'type': 'models', 'url': 'https://huggingface.co/models'}, {'type': 'datasets', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces', 'url': 'https://huggingface.co/spaces'}, {'type': 'posts', 'url': 'https://huggingface.co/posts'}, {'type': 'enterprise', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing', 'url': 'https://huggingface.co/pricing'}, {'type': 'docs', 'url': 'https://huggingface.co/docs'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'discuss', 'url': 'https://discuss.huggingface.co'}, {'type': 'status', 'url': 'https://status.huggingface.co'}, {'type': 'GitHub', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face: Pioneering the Future of AI Collaboration

**Hugging Face, a leader in AI and machine learning solutions, continues to innovate and expand its platform for global collaboration among developers, researchers, and enterprises.**

### Introduction of Hugging Face

Founded in 2016, Hugging Face has emerged as a significant player in the AI community, offering an open-source platform for building, sharing, and collaborating on machine learning models, datasets, and applications. With over 50,000 organizations utilizing its services, the platform epitomizes the spirit of community-driven advancement in artificial intelligence.

### Recent Developments and Features

Hugging Face has recently announced several key updates that signal its commitment to enhancing user experience and functionality:

- **New AI Models:** The platform features a variety of trending models including `microsoft/phi-4` and `deepseek-ai/DeepSeek-V3`, which emphasize innovations in text generation and multimodal applications. The diversity of available models now exceeds 400,000, catering to various classifications, from computer vision to natural language processing.

- **Interactive Spaces:** Hugging Face Spaces is an innovative feature enabling users to showcase AI applications interactively. Current popular projects such as `Kokoro TTS`, which supports multiple languages, illustrate the platform's versatility in real-world applications.

- **Generative Image and Video Models:** Recent enhancements include models like `FLUX`, which allows for the creation of hand-drawn style diagrams and generates high-fidelity images from minimal inputs. This reflects Hugging Face’s broader ambition to blend user accessibility with advanced AI capabilities.

### Education and Community Engagement 

To further bolster its community, Hugging Face has introduced a **certified course on AI agents**, focusing on practical implementation using various agent frameworks. Participants can earn a certification while gaining insights into building and deploying intelligent AI systems. This educational initiative underscores the platform's dedication to sharing knowledge and fostering growth within the AI community.

### Enterprise Solutions to Boost AI Development

Hugging Face has strengthened its enterprise offerings, which now include advanced security features, dedicated support, and custom pricing plans starting at **$20 per user per month**. These services are tailored for organizations seeking robust AI solutions while ensuring collaboration remains at the forefront.

- **Pro and Enterprise Pricing:** The updated pricing model introduces a **Pro account** at **$9/month**, providing subscribers access to advanced features and prioritization in GPU usage. 

- **Compute and Data Management Tools:** Users can also leverage cost-effective compute options with specialized access to private datasets, enabling enhanced scalability for organizations looking to deploy AI solutions efficiently.

### The Future of AI Collaboration

Through its open-source ethos and community-driven projects, Hugging Face is not only reshaping how developers and researchers collaborate but is also setting industry standards in ethical AI development. With a focus on transparency and accessibility, the firm aims to democratize AI technology for individuals and enterprises alike.

Hugging Face stands out as a platform that bridges technology and community, enabling everyone from hobbyists to large enterprises to contribute to and benefit from cutting-edge machine learning advancements. As the demand for AI solutions expands, Hugging Face's commitment to innovation and collaboration positions it at the forefront of the industry.

### Conclusion

In an era where AI continues to permeate various sectors, Hugging Face's commitment to building a collaborative platform fuels exciting developments and empowers users globally, paving the way for the future of artificial intelligence. 

---

For more information on Hugging Face, visit [huggingface.co](https://huggingface.co).

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>